<a href="https://colab.research.google.com/github/Aamirrafi/Aamirrafi.github.io/blob/master/LogisticregressionUsingWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 5.4MB/s 
     |████████████████████████████████| 194kB 29.4MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 1.1MB 16.3MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 40kB 6.8MB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 2.6MB 41.8MB/s 
Mounted at /content/gdrive


In [2]:
from fastbook import *

In [2]:
!pip install --upgrade gensim
!pip install nltk
!pip install tqdm
!pip install --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-1.0.1-cp27-none-linux_x86_64.whl
!pip install keras
!pip install bokeh

     |████████████████████████████████| 24.2MB 129kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
ERROR: tensorflow-1.0.1-cp27-none-linux_x86_64.whl is not a supported wheel on this platform.


In [3]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
import os;
cwd = os.getcwd()
cwd

'/content'

In [86]:
data = pd.read_csv('/content/Ethos_Dataset_Binary (1).csv', sep='_')
data.head()

,comment;isHate
0,You should know women's sports are a joke;1.0
1,You look like Sloth with deeper Down’s syndrom...
2,You look like Russian and speak like Indian. B...
3,"Women deserve to be abused, I guess.;1.0"
4,Women are made for making babies and cooking d...


In [109]:
texts = []
labels = []
data.shape
a = data["comment;isHate"][0]
a
for _ in range(data.shape[0]):
    a = data["comment;isHate"][_].split(';')
    texts.append(a[0])
    labels.append(a[1])
df = pd.DataFrame({"Sentiment":texts, "Label":labels})
df.head()
df.shape
# df['Sentiment'][1]

(998, 2)

In [118]:
!pip install nltk
import nltk
nltk.download('punkt')

tokenList = []
from nltk.tokenize import word_tokenize
for i in range(df.shape[0]):
    df['Sentiment'][i] = word_tokenize(df['Sentiment'][i])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [119]:
df.head()

,Sentiment,Label
0,"[You, should, know, women, 's, sports, are, a,...",1.0
1,"[You, look, like, Sloth, with, deeper, Down, ’...",1.0
2,"[You, look, like, Russian, and, speak, like, I...",1.0
3,"[Women, deserve, to, be, abused, ,, I, guess, .]",1.0
4,"[Women, are, made, for, making, babies, and, c...",1.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [117]:
tokenList[2]

['You',
 'look',
 'like',
 'Russian',
 'and',
 'speak',
 'like',
 'Indian',
 '.',
 'Both',
 'are',
 'disgusting',
 'go',
 'kill',
 'yourself']

In [134]:
x_train, x_test, y_train, y_test = train_test_split(np.array(df.head(df.shape[0]).Sentiment),
                                                    np.array(df.head(df.shape[0]).Label), test_size=0.2)

In [138]:
x_train.shape
x_test.shape

(200,)

In [144]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')
x_train[0]


0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
798it [00:00, 75644.78it/s]

200it [00:00, 113790.12it/s]


LabeledSentence(words=LabeledSentence(words=LabeledSentence(words=['Just', 'shoot', 'her', ',', 'one', 'less', 'of', 'them', 'to', 'have', 'to', 'deal', 'with', 'in', 'the', 'future', '.'], tags=['TRAIN_0']), tags=['TRAIN_0']), tags=['TRAIN_0'])

In [142]:
tweet_w2v = Word2Vec(size=50, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)])


100%|██████████| 798/798 [00:00<00:00, 448967.75it/s]

100%|██████████| 798/798 [00:00<00:00, 880803.84it/s]


ValueError: ignored

In [145]:
tweet_w2v['good']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.00965062,  0.00324372, -0.00896322,  0.00421206, -0.00773875,
       -0.00712252,  0.00531275,  0.00962523,  0.00167505,  0.00667445,
       -0.00974777, -0.00087687, -0.00620776, -0.00742694,  0.00669418,
        0.00535823, -0.00802975,  0.00556224, -0.00748688, -0.00397683,
        0.00851855,  0.00734875,  0.00206363,  0.00965032,  0.00250946,
        0.00552472,  0.00066614, -0.00523246,  0.00828066,  0.00133041,
       -0.00947325, -0.00569729, -0.00567646, -0.00682425, -0.00278716,
       -0.00554775,  0.0076978 ,  0.00316488,  0.00019134, -0.00649665,
       -0.00915688,  0.0082571 ,  0.00366432,  0.00890142,  0.00902696,
       -0.00297261, -0.00029271, -0.00136625, -0.00564861,  0.00071739],
      dtype=float32)

In [155]:
tweet_w2v.most_similar('a')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('very', 0.41177457571029663),
 ('male', 0.38833317160606384),
 ('&', 0.3802075684070587),
 ('down', 0.3518092632293701),
 ("'", 0.316259503364563),
 ('talk', 0.2954102158546448),
 ('its', 0.29250580072402954),
 ('were', 0.28576916456222534),
 ('They', 0.2844737768173218),
 ('also', 0.2747310400009155)]